## Working With Gradio

UI using Gradio

In [15]:
import os 
from dotenv import load_dotenv
from openai import OpenAI

In [ ]:
#import gradio
import gradio as gr

In [ ]:
# Load environment variables in a file called .env
# Print the key prefixes to help with any debugging
# You can choose whichever providers you like - or all Ollama

load_dotenv(override=True)
openai_api_key = os.getenv('OPENAI_API_KEY')

if openai_api_key:
    print(f"OpenAI API Key exists and begins {openai_api_key[:8]}")
else:
    print("OpenAI API Key not set")

In [16]:
# Connect to OpenAI and ollama
ollama_base_url = "http://localhost:11434/v1" 

openai = OpenAI()
ollama = OpenAI(base_url=ollama_base_url, api_key="ollama")

In [5]:
system_prompt = "Yo are a helpful assistant"

def message_gpt(prompt):
    messages = [{"role": "system", "content": system_prompt}, {"role": "user", "content": prompt}]
    response = openai.chat.completions.create(model="gpt-4.1-mini", messages=messages)
    return response.choices[0].message.content

In [ ]:
# This can reveal the "training cut off", or the most recent date in the training data
message_gpt("What is today's date?")

## Building a simple User Interface

In [11]:
#Siple function
def shout(text):
    print(f"Shout has been called with input {text}")
    return text.upper()

In [ ]:
shout("hello")

In [ ]:
gr.Interface(fn=shout, inputs="textbox", outputs="textbox", flagging_mode="never").launch()

In [ ]:
# Adding share=True means that it can be accessed publically
# A more permanent hosting is available using a platform called Spaces from HuggingFace, which we will touch on next week
# NOTE: Some Anti-virus software and Corporate Firewalls might not like you using share=True. 
# If you're at work on on a work network, I suggest skip this test.
gr.Interface(fn=shout, inputs="textbox", outputs="textbox", flagging_mode="never").launch(share=True)

In [ ]:
#To open ina browser 
gr.Interface(fn=shout, inputs="textbox", outputs="textbox", flagging_mode="never").launch(inbrowser=True)

In [ ]:
#To add authentication
gr.Interface(fn=shout, inputs="textbox", outputs="textbox", flagging_mode="never").launch(inbrowser=True, auth=("user", "1234"))

In [ ]:
# Define this variable and then pass js=force_dark_mode when creating the Interface
force_dark_mode = """
function refresh() {
    const url = new URL(window.location);
    if (url.searchParams.get('__theme') !== 'dark') {
        url.searchParams.set('__theme', 'dark');
        window.location.href = url.href;
    }
}
"""
gr.Interface(fn=shout, inputs="textbox", outputs="textbox", flagging_mode="never", js=force_dark_mode).launch()

In [ ]:
#Customizing gradio a bit
message_input = gr.Textbox(label="Your message", info="Enter a message to be shouted", lines=7)
message_output = gr.Textbox(label="Response", lines=8)

view = gr.Interface(
    fn=shout,
    title="Shout",
    inputs=[message_input],
    outputs=[message_output],
    examples=["hello", "hi"],
    flagging_mode="never"
    )
view.launch()

In [ ]:
#Using llm call in gradio
message_input = gr.Textbox(label="Your message", info="Enter a message to be shouted", lines=7)
message_output = gr.Textbox(label="Response", lines=8)

view = gr.Interface(
    fn=message_gpt,
    title="GPT",
    inputs=[message_input],
    outputs=[message_output],
    examples=["hello", "hi"],
    flagging_mode="never"
    )
view.launch()

In [ ]:
#Making reposne in Markdown
system_prompt = "Your Ai assitant who replies in a Hilarious way"

message_input = gr.Textbox(label="Your message", info="Enter a message to be shouted", lines=7)
message_output = gr.Markdown(label="Response")

view = gr.Interface(
    fn=message_gpt,
    title="GPT",
    inputs=[message_input],
    outputs=[message_output],
    examples=[
        "Tell me about AI", 
        "Tell me about AGI"
        ],
    flagging_mode="never"
    )
view.launch()

In [8]:
#Add streaming results
def stream_gpt(prompt):
    messages = [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": prompt}
    ]
    stream = openai.chat.completions.create(
        model="gpt-4.1-mini",
        messages=messages,
        stream= True
    )
    result = ""
    for chunk in stream:
        result += chunk.choices[0].delta.content or ""
        yield result

In [ ]:
#Making reposne in Markdown
system_prompt = "Your Ai assitant who replies in Markdown without code blocks"

message_input = gr.Textbox(label="Your message", info="Enter a message to be shouted", lines=7)
message_output = gr.Markdown(label="Response")

view = gr.Interface(
    fn=stream_gpt,
    title="GPT",
    inputs=[message_input],
    outputs=[message_output],
    examples=[
        "Tell me about AI", 
        "Tell me about AGI"
        ],
    flagging_mode="never"
    )
view.launch()

In [9]:
#Add streaming results
def stream_gpt_nano(prompt):
    messages = [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": prompt}
    ]
    stream = openai.chat.completions.create(
        model="gpt-5-nano",
        messages=messages,
        stream= True
    )
    result = ""
    for chunk in stream:
        result += chunk.choices[0].delta.content or ""
        yield result

In [ ]:
#Making reposne in Markdown ollama
system_prompt = "Your Ai assitant who replies in Markdown without code blocks"

message_input = gr.Textbox(label="Your message", info="Enter a message to be shouted", lines=7)
message_output = gr.Markdown(label="Response")

view = gr.Interface(
    fn=stream_gpt_nano,
    title="GPT",
    inputs=[message_input],
    outputs=[message_output],
    examples=[
        "Tell me about AI", 
        "Tell me about AGI"
        ],
    flagging_mode="never"
    )
view.launch()

## Using Multiple models

In [46]:
def stream_model(prompt, model):
    if model=="GPT-MINI":
        result = stream_gpt(prompt)
    elif model=="GPT-NANO":
        result = stream_gpt_nano(prompt)
    else:
        raise ValueError("Unknown model")
    yield from result

In [ ]:
message_input = gr.Textbox(label="Your message:", info="Enter a message for the LLM", lines=7)
model_selector = gr.Dropdown(["GPT-MINI", "GPT-NANO"], label="Select model", value="GPT")
message_output = gr.Markdown(label="Response:")

view = gr.Interface(
    fn=stream_model,
    title="LLMs", 
    inputs=[message_input, model_selector], 
    outputs=[message_output], 
    examples=[
            ["Explain the Transformer architecture to a layperson", "GPT-MINI"],
            ["Explain the Transformer architecture to an aspiring AI engineer", "GPT-NANO"]
        ], 
    flagging_mode="never"
    )
view.launch()

# Building a company brochure generator

In [1]:
from scraper import fetch_website_contents

d:\Learning\Udemy Course projects\Ai Engineering core track - Ed Sonner\LLM_Engineering_Practice\.venv\Lib\site-packages\requests\__init__.py:86: RequestsDependencyWarning: Unable to find acceptable character detection dependency (chardet or charset_normalizer).
  warnings.warn(


In [2]:

# Changing the global variable we used above:
system_message = """
You are an assistant that analyzes the contents of a company website landing page
and creates a short brochure about the company for prospective customers, investors and recruits.
Respond in markdown without code blocks.
"""

In [11]:
def stream_brochure(company_name, url, model):
    yield ""
    prompt = f"Please generate a company brochure for {company_name}. Here is their landing page:\n"
    prompt += fetch_website_contents(url)
    if model=="GPT-MINI":
        result = stream_gpt(prompt)
    elif model=="GPT-NANO":
        result = stream_gpt_nano(prompt)
    else:
        raise ValueError("Unknown model")
    yield from result

In [ ]:
company_name = gr.Textbox(label="Company Name")
url_link = gr.Textbox(label="Landing page URL including http:// or https://")
model_name = gr.Dropdown(["GPT-MINI", "GPT-NANO"], label="Model", value="GPT_MINI")
result = gr.Markdown(label="Response: ")

view = gr.Interface(
    fn=stream_brochure,
    title="Brochure Generator",
    inputs=[company_name,url_link,model_name],
    outputs=[result],
    examples=[
            ["Hugging Face", "https://huggingface.co", "GPT-MINI"],
            ["Edward Donner", "https://edwarddonner.com", "GPT-NANO"]
    ],
    flagging_mode="never" 
)
view.launch()